   ### Capstone Project: The Battle of Neighbourhoods Indian Restaurants in Berlin, German


### IBM Data Science Course
### Coursera Capstone Project
### The battle of neighborhoods

# Step 1: install required libraries


In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('All libraries imported.')

All libraries imported.


# Step 2: Padova/Padua
##### Padova/Padua: residence location and venues (source: FourSquare)

In [2]:
# Via Manzoni, Padova, Italy
address = 'Via Manzoni, Padova, Italy'
geolocator = Nominatim(user_agent="thebofn")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
neighborhood_latitude=45.3932529
neighborhood_longitude=11.8799238
print('The geograpical coordinates of my Padova home are {}, {}.'.format(neighborhood_latitude, neighborhood_longitude))

The geograpical coordinates of my Padova home are 45.3932529, 11.8799238.


Start with Park and playground

In [3]:
CLIENT_ID = 'DHQVKXTSH4R2LAKLZTI3NS5ZGHUQOBXLPD4SSEYJYSUYJ5KF' # your Foursquare ID
CLIENT_SECRET = 'WM3FT0GHS5SFU2FME1JCFJARPJQJLXXEJP2LVUMDKLUX0PHB' # your Foursquare Secret
VERSION = '20190801' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# CATEGORIES I AM INTERESTED ARE (TO FILTER THE SEARCH)
# PARK 4bf58dd8d48988d163941735,5bae9231bedf3950379f89d0,52e81612bcbc57f1066b7a21,52e81612bcbc57f1066b7a13,4bf58dd8d48988d162941735
# PLAYGROUND 4bf58dd8d48988d1e7941735
categoryId='4bf58dd8d48988d163941735,5bae9231bedf3950379f89d0,52e81612bcbc57f1066b7a21,52e81612bcbc57f1066b7a13,4bf58dd8d48988d162941735'

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    categoryId,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DHQVKXTSH4R2LAKLZTI3NS5ZGHUQOBXLPD4SSEYJYSUYJ5KF&client_secret=WM3FT0GHS5SFU2FME1JCFJARPJQJLXXEJP2LVUMDKLUX0PHB&v=20190801&ll=45.3932529,11.8799238&categoryId=4bf58dd8d48988d163941735,5bae9231bedf3950379f89d0,52e81612bcbc57f1066b7a21,52e81612bcbc57f1066b7a13,4bf58dd8d48988d162941735&radius=1000&limit=100'

In [4]:
results = requests.get(url).json()
#results (to keep clean I commented it)

In [5]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
venues = results['response']['groups'][0]['items']
PDnearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
PDnearby_venues = PDnearby_venues.loc[:, filtered_columns]
# filter the category for each row
PDnearby_venues['venue.categories'] = PDnearby_venues.apply(get_category_type, axis=1)
# clean columns
PDnearby_venues.columns = [col.split(".")[-1] for col in PDnearby_venues.columns]

PDnearby_venues.shape

(5, 4)

In [7]:
# Venues near my current Padova/Padua home
PDnearby_venues['categories'].value_counts()

Other Great Outdoors    2
Garden                  1
Plaza                   1
Park                    1
Name: categories, dtype: int64

In [8]:
PDnearby_venues


,name,categories,lat,lng
0,Prato della Valle,Plaza,45.398420,11.876521
1,Orto Botanico (Orto botanico di Padova),Garden,45.399007,11.879603
2,Centro Giovanile Antonianum,Other Great Outdoors,45.398539,11.879618
3,Giardini Santa Rita,Park,45.391702,11.888398
4,Via Umberto I,Other Great Outdoors,45.401423,11.875735


In [9]:
map_pd = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)
# add markers to map
for lat, lng, label in zip(PDnearby_venues['lat'], PDnearby_venues['lng'], PDnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_pd)  
    
map_pd

We continue with schools (primary, middle and high schools)

In [10]:
CLIENT_ID = 'DHQVKXTSH4R2LAKLZTI3NS5ZGHUQOBXLPD4SSEYJYSUYJ5KF' # your Foursquare ID
CLIENT_SECRET = 'WM3FT0GHS5SFU2FME1JCFJARPJQJLXXEJP2LVUMDKLUX0PHB' # your Foursquare Secret
VERSION = '20190801' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
# CATEGORIES I AM INTERESTED ARE (TO FILTER THE SEARCH)
# SCHOOLS Primary 4f4533804b9074f6e4fb0105, Middle 4f4533814b9074f6e4fb0106, High 4bf58dd8d48988d13d941735
categoryId='4f4533804b9074f6e4fb0105,4f4533814b9074f6e4fb0106,4bf58dd8d48988d13d941735'

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    categoryId,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=DHQVKXTSH4R2LAKLZTI3NS5ZGHUQOBXLPD4SSEYJYSUYJ5KF&client_secret=WM3FT0GHS5SFU2FME1JCFJARPJQJLXXEJP2LVUMDKLUX0PHB&v=20190801&ll=45.3932529,11.8799238&categoryId=4f4533804b9074f6e4fb0105,4f4533814b9074f6e4fb0106,4bf58dd8d48988d13d941735&radius=1000&limit=100'

In [11]:
results = requests.get(url).json()
#results (to keep clean I commented it)
venues = results['response']['groups'][0]['items']
PDnearby_venues2 = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
PDnearby_venues2 = PDnearby_venues2.loc[:, filtered_columns]
# filter the category for each row
PDnearby_venues2['venue.categories'] = PDnearby_venues2.apply(get_category_type, axis=1)
# clean columns
PDnearby_venues2.columns = [col.split(".")[-1] for col in PDnearby_venues2.columns]

PDnearby_venues2.shape

(3, 4)

In [12]:
PDnearby_venues2


,name,categories,lat,lng
0,Itis Ruzza,High School,45.399522,11.882700
1,Liceo Scientifico E. Fermi,High School,45.397916,11.873107
2,ITC Calvi,High School,45.401920,11.880880


In [13]:
map_pd = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)
# add markers to map
for lat, lng, label in zip(PDnearby_venues2['lat'], PDnearby_venues2['lng'], PDnearby_venues2['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_pd)  
    
map_pd

We continue with shopping venues

In [14]:
results = requests.get(url).json()
#results (to keep clean I commented it)
venues = results['response']['groups'][0]['items']
PDnearby_venues3 = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
PDnearby_venues3 = PDnearby_venues3.loc[:, filtered_columns]
# filter the category for each row
PDnearby_venues3['venue.categories'] = PDnearby_venues3.apply(get_category_type, axis=1)
# clean columns
PDnearby_venues3.columns = [col.split(".")[-1] for col in PDnearby_venues3.columns]

PDnearby_venues3.shape

(3, 4)

In [15]:
PDnearby_venues3


,name,categories,lat,lng
0,Itis Ruzza,High School,45.399522,11.882700
1,Liceo Scientifico E. Fermi,High School,45.397916,11.873107
2,ITC Calvi,High School,45.401920,11.880880


In [16]:
map_pd = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)
# add markers to map
for lat, lng, label in zip(PDnearby_venues3['lat'], PDnearby_venues3['lng'], PDnearby_venues3['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_pd)  
    
map_pd

arts and entertainment venues

In [17]:
results = requests.get(url).json()
#results (to keep clean I commented it)
venues = results['response']['groups'][0]['items']
PDnearby_venues4 = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
PDnearby_venues4 = PDnearby_venues4.loc[:, filtered_columns]
# filter the category for each row
PDnearby_venues4['venue.categories'] = PDnearby_venues4.apply(get_category_type, axis=1)
# clean columns
PDnearby_venues4.columns = [col.split(".")[-1] for col in PDnearby_venues4.columns]

PDnearby_venues4.shape

(3, 4)

In [18]:
PDnearby_venues4


,name,categories,lat,lng
0,Itis Ruzza,High School,45.399522,11.882700
1,Liceo Scientifico E. Fermi,High School,45.397916,11.873107
2,ITC Calvi,High School,45.401920,11.880880


In [19]:
map_pd = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)
# add markers to map
for lat, lng, label in zip(PDnearby_venues4['lat'], PDnearby_venues4['lng'], PDnearby_venues4['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=7,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_pd)  
    
map_pd

In [20]:
PDnearby_venues_tot = pd.concat([PDnearby_venues, PDnearby_venues2, PDnearby_venues3, PDnearby_venues4])
PDnearby_venues_tot.shape

(14, 4)

In [21]:
PDnearby_venues.head()


,name,categories,lat,lng
0,Prato della Valle,Plaza,45.398420,11.876521
1,Orto Botanico (Orto botanico di Padova),Garden,45.399007,11.879603
2,Centro Giovanile Antonianum,Other Great Outdoors,45.398539,11.879618
3,Giardini Santa Rita,Park,45.391702,11.888398
4,Via Umberto I,Other Great Outdoors,45.401423,11.875735


In [22]:
PDnearby_venues_tot.tail()


,name,categories,lat,lng
1,Liceo Scientifico E. Fermi,High School,45.397916,11.873107
2,ITC Calvi,High School,45.401920,11.880880
0,Itis Ruzza,High School,45.399522,11.882700
1,Liceo Scientifico E. Fermi,High School,45.397916,11.873107
2,ITC Calvi,High School,45.401920,11.880880


# Step 3: Las Vegas

In [23]:
lvneighborhoods = pd.read_csv("LV_neighborhoods.csv",sep=';',usecols=['City/CDP', 'Neighborhood', 'Latitude', 'Longitude'] ) 
lvneighborhoods

FileNotFoundError: [Errno 2] File b'LV_neighborhoods.csv' does not exist: b'LV_neighborhoods.csv'

In [ ]:
# create map of Las Vegas using latitude and longitude values
map_lv = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, citycdp, neighborhood in zip(lvneighborhoods['Latitude'], lvneighborhoods['Longitude'], lvneighborhoods['City/CDP'], lvneighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, citycdp)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lv)  
    
map_lv

### Las Vegas neighborhoods venues (source Foursquare API)

In [ ]:
CLIENT_ID = 'DHQVKXTSH4R2LAKLZTI3NS5ZGHUQOBXLPD4SSEYJYSUYJ5KF' # my Foursquare ID
CLIENT_SECRET = 'WM3FT0GHS5SFU2FME1JCFJARPJQJLXXEJP2LVUMDKLUX0PHB' # my Foursquare Secret
VERSION = '20190730' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # CATEGORIES I AM INTERESTED ARE (TO FILTER THE SEARCH)
        # PARK 4bf58dd8d48988d163941735,5bae9231bedf3950379f89d0,52e81612bcbc57f1066b7a21,52e81612bcbc57f1066b7a13,4bf58dd8d48988d162941735
        # PLAYGROUND 4bf58dd8d48988d1e7941735
        categoryId='4bf58dd8d48988d163941735,5bae9231bedf3950379f89d0,52e81612bcbc57f1066b7a21,52e81612bcbc57f1066b7a13,4bf58dd8d48988d162941735'

        LIMIT = 100 # limit of number of venues returned by Foursquare API
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            categoryId,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)                             

In [ ]:
lv_venues = getNearbyVenues(names=lvneighborhoods['Neighborhood'],
                                   latitudes=lvneighborhoods['Latitude'],
                                   longitudes=lvneighborhoods['Longitude']
                                  )
print(lv_venues.shape)
lv_venues.head()

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # CATEGORIES I AM INTERESTED ARE (TO FILTER THE SEARCH)
        # SCHOOLS Primary 4f4533804b9074f6e4fb0105, Middle 4f4533814b9074f6e4fb0106, High 4bf58dd8d48988d13d941735
        categoryId='4f4533804b9074f6e4fb0105,4f4533814b9074f6e4fb0106,4bf58dd8d48988d13d941735'
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            categoryId,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
lv_venues2 = getNearbyVenues(names=lvneighborhoods['Neighborhood'],
                                   latitudes=lvneighborhoods['Latitude'],
                                   longitudes=lvneighborhoods['Longitude']
                                  )
print(lv_venues2.shape)
lv_venues2.head()

In [ ]:
lv_venues_tot = pd.concat([lv_venues, lv_venues2, lv_venues3, lv_venues4])
lv_venues_tot.shape

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = lv_grouped['Neighborhood']

for ind in np.arange(lv_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(lv_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

lv_grouped_clustering = lv_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lv_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

lv_merged = lvneighborhoods

# merge lv_grouped with lv_data to add latitude/longitude for each neighborhood
lv_merged = lv_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

lv_merged = lv_merged.dropna().astype({"Cluster Labels": int})

lv_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lv_merged['Latitude'], lv_merged['Longitude'], lv_merged['Neighborhood'], lv_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
lv_merged.loc[lv_merged['Cluster Labels'] == 0, lv_merged.columns[[1] + list(range(5, lv_merged.shape[1]))]]

In [ ]:
lv_merged.loc[lv_merged['Cluster Labels'] == 1, lv_merged.columns[[1] + list(range(5, lv_merged.shape[1]))]]


In [ ]:
lv_merged.loc[lv_merged['Cluster Labels'] == 2, lv_merged.columns[[1] + list(range(5, lv_merged.shape[1]))]]


In [ ]:
lv_merged.loc[lv_merged['Cluster Labels'] == 3, lv_merged.columns[[1] + list(range(5, lv_merged.shape[1]))]]


In [ ]:
lv_merged.loc[lv_merged['Cluster Labels'] == 3, lv_merged.columns[[1] + list(range(5, lv_merged.shape[1]))]]


In [ ]:
lvhouses = pd.read_csv("LV_Houses.csv",sep=';') 
lvhouses.head()

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # CATEGORIES I AM INTERESTED ARE (TO FILTER THE SEARCH)
        # PARK 4bf58dd8d48988d163941735,5bae9231bedf3950379f89d0,52e81612bcbc57f1066b7a21,52e81612bcbc57f1066b7a13,4bf58dd8d48988d162941735
        # PLAYGROUND 4bf58dd8d48988d1e7941735
        categoryId='4bf58dd8d48988d163941735,5bae9231bedf3950379f89d0,52e81612bcbc57f1066b7a21,52e81612bcbc57f1066b7a13,4bf58dd8d48988d162941735'

        LIMIT = 100 # limit of number of venues returned by Foursquare API
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            categoryId,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['House', 
                  'House Latitude', 
                  'House Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)            

In [ ]:
lv_h_venues = getNearbyVenues(names=lvhouses['ID_House'],
                                   latitudes=lvhouses['Latitude'],
                                   longitudes=lvhouses['Longitude']
                                  )
lv_h_venues.sort_values(by=['House'])

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # CATEGORIES I AM INTERESTED ARE (TO FILTER THE SEARCH)
        # SCHOOLS Primary 4f4533804b9074f6e4fb0105, Middle 4f4533814b9074f6e4fb0106, High 4bf58dd8d48988d13d941735
        categoryId='4f4533804b9074f6e4fb0105,4f4533814b9074f6e4fb0106,4bf58dd8d48988d13d941735'

        LIMIT = 100 # limit of number of venues returned by Foursquare API
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            categoryId,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['House', 
                  'House Latitude', 
                  'House Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
            

In [ ]:
lv_h2_venues = getNearbyVenues(names=lvhouses['ID_House'],
                                   latitudes=lvhouses['Latitude'],
                                   longitudes=lvhouses['Longitude']
                                  )
lv_h2_venues.sort_values(by=['House'])

### Shopping venues

In [ ]:
Shopping venues
In [116]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # CATEGORIES I AM INTERESTED ARE (TO FILTER THE SEARCH)
        # SHOPPING 4d4b7105d754a06378d81259
        categoryId='4d4b7105d754a06378d81259'   

        LIMIT = 100 # limit of number of venues returned by Foursquare API
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            categoryId,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['House', 
                  'House Latitude', 
                  'House Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)        

In [ ]:
lv_h3_venues = getNearbyVenues(names=lvhouses['ID_House'],
                                   latitudes=lvhouses['Latitude'],
                                   longitudes=lvhouses['Longitude']
                                  )
lv_h3_venues.House.value_counts()

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # CATEGORIES I AM INTERESTED ARE (TO FILTER THE SEARCH)
        # ARTS AND THEATRES 4d4b7104d754a06370d81259
        categoryId='4d4b7104d754a06370d81259'

        LIMIT = 100 # limit of number of venues returned by Foursquare API
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            categoryId,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['House', 
                  'House Latitude', 
                  'House Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)        

In [ ]:
lv_h4_venues = getNearbyVenues(names=lvhouses['ID_House'],
                                   latitudes=lvhouses['Latitude'],
                                   longitudes=lvhouses['Longitude']
                                  )
lv_h4_venues.sort_values(by=['House'])


# Step 4: Results


In [ ]:
# create map of Las Vegas 
latitude= 36.1662859
longitude= -115.149225

map_lv_one = folium.Map(location=[latitude, longitude], zoom_start=10)

# add houses markers to map
for lat, lng, label in zip(lvhouses['Latitude'], lvhouses['Longitude'],lvhouses['Address']):      
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lv_one) 

data = [['Caesars Palace Casino', '36.1161685', '-115.1766877']] 
lv_job_location = pd.DataFrame(data, columns = ['Name', 'Latitude', 'Longitude']) 

# add job location (Caesars palace casino) marker to map
for lat, lng, label in zip(lv_job_location['Latitude'], lv_job_location['Longitude'],lv_job_location['Name']):      
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='green',
        fill=True,
        fill_color='#6bef7f',
        fill_opacity=0.7,
        parse_html=False).add_to(map_lv_one) 
    
# set color scheme for the clusters
kclusters=5
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lv_merged['Latitude'], lv_merged['Longitude'], lv_merged['Neighborhood'], lv_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=15,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_lv_one)

# adds tool to the top right
from folium.plugins import MeasureControl
map_lv_one.add_child(MeasureControl())

# measurement ruler icon tool to measure distances in map
from folium.plugins import FloatImage
url = ('https://media.licdn.com/mpr/mpr/shrinknp_100_100/AAEAAQAAAAAAAAlgAAAAJGE3OTA4YTdlLTkzZjUtNDFjYy1iZThlLWQ5OTNkYzlhNzM4OQ.jpg')
FloatImage(url, bottom=5, left=85).add_to(map_lv_one)

map_lv_one


# Step 5: Problem resolution


In [ ]:
## cn is the cluster number to explore
cn = 1
lv_merged.loc[lv_merged['Cluster Labels'] == cn, lv_merged.columns[[1] + list(range(5, lv_merged.shape[1]))]

# Step 6: Conclusions


I can conclude that venues in House #2 area are pretty much the same like in my Padova/Padua home area even more shopping venues for my wife happiness. It is not far from job location and in a good neighborhood.
House #2 is a perfect fit for me and my family.